In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import mplfinance as mpf
import time
import settrade_v2
from settrade_v2 import Investor


In [2]:
# เชื่อมต่อกับ Cassandra Docker
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()


In [3]:
keyspace_name = 'stock_data'

session.execute(f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH REPLICATION = {{ 'class': 'SimpleStrategy', 'replication_factor': 1 }};")
session.set_keyspace(keyspace_name)

In [4]:
# ดึง keyspace ทั้งหมด
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")

# แสดงผลลัพธ์
print("Available Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

Available Keyspaces:
- stock_technical
- system_auth
- system_schema
- system_distributed
- system
- system_traces
- stock_data


In [5]:
insert_query = session.prepare("""
    INSERT INTO stock_price (symbol, date, open, high, low, close, volume)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""")

In [6]:

investor = Investor(
    app_id="ZGCsmM0t5c9VMLRJ",                                 
    app_secret="T5eQ70WLmOWDpVvV7jicJtspuBR2A361nzVGmR8xTF4=", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)

deri = investor.Derivatives(account_no="Hilman-D")
account_info = deri.get_account_info()

account_info 

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [7]:
# Get real-time quote
market_data = investor.MarketData()
stock = market_data.get_quote_symbol("AOT")
print(stock)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 0, 'totalSellVolume': 0, 'totalNoSideVolume': 0, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': 0.72783, 'pe': 25.81, 'pbv': 4.03, 'percentYield': 2.2699999809265137, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [8]:
# get candlestick
market_cd = investor.MarketData()
res = market_cd.get_candlestick(symbol="AOT", interval="1d",normalized=True,)

In [9]:
# 👀 ตรวจสอบก่อนว่า res เป็น dict และมี key 'data' หรือไม่
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,0,1682010000,72.25,72.50,71.50,72.25,15866323,1.143215e+09
1,0,1682269200,72.00,72.75,72.00,72.75,13050472,9.451594e+08
2,0,1682355600,72.50,72.75,72.25,72.25,15141138,1.098014e+09
3,0,1682442000,72.50,73.00,72.50,73.00,10924024,7.954887e+08
4,0,1682528400,73.25,73.50,72.75,73.00,12732294,9.303291e+08
...,...,...,...,...,...,...,...,...
495,0,1746464400,38.75,39.00,37.00,37.00,42205781,1.590734e+09
496,0,1746550800,37.25,39.50,37.25,39.25,57889130,2.237424e+09
497,0,1746637200,39.00,39.25,38.50,38.50,33155908,1.284850e+09
498,0,1746723600,39.00,39.25,38.25,39.00,27097430,1.048902e+09


In [10]:
df.columns

Index(['lastSequence', 'time', 'open', 'high', 'low', 'close', 'volume',
       'value'],
      dtype='object')

In [11]:
# แปลงวันที่ให้เป็น datetime object
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

# วาดกราฟ
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Candlestick"
))

fig.add_trace(go.Bar(
    x=df.index,
    y=df["volume"],
    name="Volume",
    yaxis='y2',
    marker_color='rgba(150,150,150,0.3)'
))

fig.update_layout(
    title="กราฟแท่งเทียนหุ้น (จาก Settrade API)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="ราคา"),
    yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False),
    height=600,
)

fig.show()

In [12]:
session.set_keyspace('stock_data')

In [13]:
#GetQuoteSymbol
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("AOT")
print(res)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 0, 'totalSellVolume': 0, 'totalNoSideVolume': 0, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': 0.72783, 'pe': 25.81, 'pbv': 4.03, 'percentYield': 2.2699999809265137, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [14]:

def to_date(val):
    try:
        return datetime.strptime(val, "%Y-%m-%d").date() if val else None
    except Exception:
        return None

try:
    session.execute(insert_query, (
        res.get("symbol"),
        datetime.now().date(),  # ใช้วันที่ปัจจุบันหรือกำหนดเอง
        res.get("open") if res.get("open") is not None else res.get("last"),
        res.get("high"),
        res.get("low"),
        res.get("close") if res.get("close") is not None else res.get("last"),
        int(res.get("totalVolume")) if res.get("totalVolume") is not None else 0
    ))
    print("✅ Inserted AOT data ")
except Exception as e:
    print(f"❌ Error inserting AOT data: {e}")


✅ Inserted AOT data 


In [15]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name} LIMIT 10")
df = pd.DataFrame(rows)
df.head(10)

,symbol,date,close,high,low,open,volume
0,PPPM,2025-05-16,0.41,0.43,0.38,0.41,780900
1,TPCH,2025-05-16,2.78,2.80,2.72,2.78,258400
2,KPNREIT,2025-05-16,NaN,NaN,NaN,NaN,0
3,POLY,2025-05-16,6.70,6.70,6.70,6.70,1100
4,QHBREIT,2025-05-16,3.62,3.62,3.58,3.62,240700
5,VCOM,1970-01-01,2.40,2.44,2.38,2.44,152327
6,VCOM,2025-05-16,2.46,2.50,2.44,2.46,43500
7,KDH,2025-05-16,NaN,NaN,NaN,NaN,0
8,NVD,2025-05-16,1.10,1.16,1.10,1.10,16900
9,JDF,2025-05-16,1.85,1.98,1.85,1.85,78600


In [16]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.shape

(1111, 7)

In [17]:
df.head()


,symbol,date,close,high,low,open,volume
0,PPPM,2025-05-16,0.41,0.43,0.38,0.41,780900
1,TPCH,2025-05-16,2.78,2.80,2.72,2.78,258400
2,KPNREIT,2025-05-16,NaN,NaN,NaN,NaN,0
3,POLY,2025-05-16,6.70,6.70,6.70,6.70,1100
4,QHBREIT,2025-05-16,3.62,3.62,3.58,3.62,240700


In [18]:
from datetime import datetime
import pandas as pd
import time

# โหลดรายชื่อหุ้น
try:
    symbols_df = pd.read_excel("/Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/data_collection/settrade1.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ดึงข้อมูลหุ้น
def fetch_stock_data(symbol, market_data):
    try:
        res = market_data.get_quote_symbol(symbol)
        if not isinstance(res, dict):
            print(f"⚠️ ไม่พบข้อมูลสำหรับ {symbol}")
            return None

        return {
            "symbol": res.get("symbol"),
            "date": datetime.now().date(),
            "open": res.get("open") or res.get("last"),
            "high": res.get("high"),
            "low": res.get("low"),
            "close": res.get("close") or res.get("last"),
            "volume": int(res.get("totalVolume") or 0)
        }
    except Exception as e:
        print(f"❌ Error fetching {symbol}: {e}")
        return None

# --- แปลงข้อมูลเป็น DataFrame ---
all_data = []

for symbol in symbols:
    print(f"📦 กำลังโหลด {symbol} ...")
    stock_data = fetch_stock_data(symbol, market_data)
    if stock_data:
        all_data.append(stock_data)
    time.sleep(2)

df = pd.DataFrame(all_data)



📦 กำลังโหลด 24CS ...
📦 กำลังโหลด 2S ...
📦 กำลังโหลด 3BBIF ...
📦 กำลังโหลด A ...
📦 กำลังโหลด A5 ...
📦 กำลังโหลด AAI ...
📦 กำลังโหลด AAV ...


KeyboardInterrupt: 

In [19]:
print(df.head())

    symbol        date  close  high   low  open  volume
0     PPPM  2025-05-16   0.41  0.43  0.38  0.41  780900
1     TPCH  2025-05-16   2.78  2.80  2.72  2.78  258400
2  KPNREIT  2025-05-16    NaN   NaN   NaN   NaN       0
3     POLY  2025-05-16   6.70  6.70  6.70  6.70    1100
4  QHBREIT  2025-05-16   3.62  3.62  3.58  3.62  240700


In [ ]:
def insert_bulk_data(df, session):
    insert_query = session.prepare("""
        INSERT INTO stock_price (symbol, date, open, high, low, close, volume)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """)
    for _, row in df.iterrows():
        session.execute(insert_query, (
            row['symbol'],
            row['date'],
            row['open'],
            row['high'],
            row['low'],
            row['close'],
            row['volume']
        ))
    print("✅ Insert ข้อมูลทั้งหมดเรียบร้อยแล้ว")

insert_bulk_data(df, session)


In [ ]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.head()

,symbol,date,close,high,low,open,volume
0,PPPM,2025-05-16,0.41,0.43,0.38,0.41,780900
1,TPCH,2025-05-16,2.78,2.80,2.72,2.78,258400
2,KPNREIT,2025-05-16,NaN,NaN,NaN,NaN,0
3,POLY,2025-05-16,6.70,6.70,6.70,6.70,1100
4,QHBREIT,2025-05-16,3.62,3.62,3.58,3.62,240700


In [20]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.shape

(1111, 7)